In [77]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [78]:
## Data Preprocessing
from sudachipy import dictionary
tokenizer = dictionary.Dictionary().create()

SOS_token = 0
EOS_token = 1
UNK_token = 2
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS":0,"EOS":1,"UNK":2}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2:"UNK"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence,type="eng"):
        if type == "eng":
            for word in sentence.split(' '):
                self.addWord(word)
        elif type=="jpn":
            tokens = [m.surface() for m in tokenizer.tokenize(sentence)]
            for token in tokens:
                self.addWord(token)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [79]:
## Some filters for English dataset; But I dont know japanese!!!!!

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [80]:
## Make the vocabulary and do some statistics

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    # lines = []
    with open('eng_jpn.txt', 'r', encoding='utf-8') as f:
        lines = f.readlines()
    pairs = []
    # Split every line into pairs and normalize
    for line in lines:
        pair_raw = line.strip().split('\t')
        pair_raw[1] = normalizeString(pair_raw[1])
        # print(pair_raw)
        pairs.append(pair_raw)
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    # pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0],"jpn")
        output_lang.addSentence(pair[1],"eng")
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)

    return input_lang, output_lang, pairs


In [ ]:

input_lang, output_lang, pairs = prepareData('Jpn', 'Eng')
print(random.choice(pairs))

In [82]:
# make the train/valid/test dataloader

def indexesFromSentence(lang, sentence,type="jpn"):
    if type == "eng":
        return [lang.word2index[word] for word in sentence.split(' ')]
    elif type=="jpn":
        tokens = [m.surface() for m in tokenizer.tokenize(sentence)]
        return [lang.word2index[token] if token in lang.word2index else lang.word2index["UNK"] for token in tokens]
    # return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence,type="jpn"):
    indexes = indexesFromSentence(lang, sentence,type)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0],"jpn")
    target_tensor = tensorFromSentence(output_lang, pair[1],"eng")
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('jpn', 'eng')

    valid_input_ids = []
    valid_target_ids = []

    for inp, tgt in pairs:
        inp_ids = indexesFromSentence(input_lang, inp, "jpn")
        tgt_ids = indexesFromSentence(output_lang, tgt, "eng")
        
        if len(inp_ids) > MAX_LENGTH - 1 or len(tgt_ids) > MAX_LENGTH - 1 or len(inp_ids) == 0 or len(tgt_ids) == 0:
            print(f"Skipped: {inp} -> {tgt}")
            continue
        # 添加 EOS_token
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        # 填充到 MAX_LENGTH
        inp_ids_padded = np.pad(inp_ids, (0, MAX_LENGTH - len(inp_ids)), mode='constant', constant_values=0)
        tgt_ids_padded = np.pad(tgt_ids, (0, MAX_LENGTH - len(tgt_ids)), mode='constant', constant_values=0)
        
        valid_input_ids.append(inp_ids_padded)
        valid_target_ids.append(tgt_ids_padded)

    valid_input_ids = np.array(valid_input_ids, dtype=np.int32)
    valid_target_ids = np.array(valid_target_ids, dtype=np.int32)

    n = len(valid_input_ids) 
    print("number of valid data: ", n)
    train_size = int(0.8 * n)
    valid_size = int(0.1 * n)
    test_size = n - train_size - valid_size 

    input_tensor = torch.LongTensor(valid_input_ids)
    target_tensor = torch.LongTensor(valid_target_ids)

    train_data = TensorDataset(input_tensor[:train_size].to(device), target_tensor[:train_size].to(device))
    valid_data = TensorDataset(input_tensor[train_size:train_size + valid_size].to(device), target_tensor[train_size:train_size + valid_size].to(device))
    test_data = TensorDataset(input_tensor[train_size + valid_size:].to(device), target_tensor[train_size + valid_size:].to(device))

    train_sampler = RandomSampler(train_data)
    valid_sampler = RandomSampler(valid_data)
    test_sampler = RandomSampler(test_data)

    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return input_lang, output_lang, train_dataloader, valid_dataloader, test_dataloader

In [83]:
## Encoder architecture
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded)  
        return output, (hidden, cell)


In [84]:
## Decoder architecture

import torch
import torch.nn as nn
import torch.nn.functional as F

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.lstm = nn.LSTM(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden  # encoder_hidden should contain (hidden, cell) for LSTM
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1)  # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        query = hidden[0].permute(1, 0, 2)  # Use hidden state for attention query, hidden[0] is the hidden state
        context, attn_weights = self.attention(query, encoder_outputs)

        input_lstm = torch.cat((embedded, context), dim=2)
        output, hidden = self.lstm(input_lstm, hidden)  # LSTM returns (output, (hidden, cell))

        output = self.out(output)
        return output, hidden, attn_weights


In [ ]:
# do it!
hidden_size = 256
batch_size = 32

input_lang, output_lang, train_dataloader, valid_dataloader , test_dataloader = get_dataloader(batch_size)

# for i, (input_tensor, target_tensor) in enumerate(train_dataloader):
#     print(input_tensor[0], target_tensor[0])
#     break

In [86]:
import torch
from tqdm import tqdm

def evaluate_epoch(dataloader, encoder, decoder, criterion):

    total_loss = 0
    with torch.no_grad():
        for data in tqdm(dataloader, desc="Evaluating", leave=False):
            input_tensor, target_tensor = data

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )

            total_loss += loss.item()
    return total_loss / len(dataloader)


def train_epoch(dataloader, valid_dataloader, encoder, decoder,
                encoder_optimizer, decoder_optimizer, criterion, epoch, best_valid_loss, ckpt_path = 'checkpoint.pth'):
    total_loss = 0
    
    # 训练模式
    encoder.train()
    decoder.train()
    
    for data in tqdm(dataloader, desc=f"Training Epoch {epoch}"):
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    # 计算训练集损失
    avg_train_loss = total_loss / len(dataloader)

    # 评估验证集损失
    avg_valid_loss = evaluate_epoch(valid_dataloader, encoder, decoder, criterion)

    print(f'Epoch {epoch}, Train Loss: {avg_train_loss:.4f}, Valid Loss: {avg_valid_loss:.4f}')

    if avg_valid_loss < best_valid_loss:
        best_valid_loss = avg_valid_loss
        torch.save({
            'epoch': epoch,
            'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),
            'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
            'loss': best_valid_loss,
        }, ckpt_path)
        print(f"Checkpoint saved at epoch {epoch} with validation loss {best_valid_loss:.4f}")
    return avg_train_loss, avg_valid_loss

In [87]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [88]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [89]:
# some evaluation methods
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [90]:
def train(train_dataloader, valid_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
          print_every=100, plot_every=100, ckpt_path='checkpoint.pth'):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    best_valid_loss = float('inf')
    
    for epoch in range(1, n_epochs + 1):
        # 训练模型并计算训练损失
        train_loss = train_epoch(train_dataloader, valid_dataloader, encoder, decoder,
                                 encoder_optimizer, decoder_optimizer, criterion, epoch, best_valid_loss, ckpt_path)

        print_loss_total += train_loss[0]  # 训练损失
        plot_loss_total += train_loss[0]    # 训练损失
        best_valid_loss = train_loss[1]
        # 计算验证损失
        valid_loss = train_loss[1]

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) Train Loss: %.4f, Valid Loss: %.4f' % (
                timeSince(start, epoch / n_epochs),
                epoch, epoch / n_epochs * 100, print_loss_avg, valid_loss))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)


In [ ]:
print(device)
# train the model!
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
print(train_dataloader)
train(train_dataloader, valid_dataloader, encoder, decoder, 10, print_every=5, plot_every=5)


In [ ]:
# after training, we can evaluate the model!
checkpoint = torch.load("checkpoint.pth")
encoder.load_state_dict(checkpoint['encoder_state_dict'])
decoder.load_state_dict(checkpoint['decoder_state_dict'])

In [ ]:
# Random evaluation
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

In [ ]:
# evaluate some specific sentences
def showAttention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))


evaluateAndShowAttention('私の名前は愛です')

evaluateAndShowAttention('昨日はお肉を食べません')

evaluateAndShowAttention('いただきますよう')

evaluateAndShowAttention('おはようございます')

evaluateAndShowAttention('秋は好きです')

In [95]:
## calculate the BLEU score
import nltk
from nltk.translate.bleu_score import corpus_bleu

def evaluate_model_on_dataloader(encoder, decoder, input_lang, output_lang, dataloader):
    hypotheses = []
    references = []

    for i, (input_tensor, target_tensor) in enumerate(dataloader):
        try:
            input_sentence = ' '.join([input_lang.index2word[idx.item()] for idx in input_tensor[0] if idx.item() != EOS_token and idx.item() != SOS_token])
            reference_sentence = ' '.join([output_lang.index2word[idx.item()] for idx in target_tensor[0] if idx.item() != EOS_token and idx.item() != SOS_token])
            reference_words = reference_sentence.split(' ')  
            reference_words.append('<EOS>')  

            output_words, _ = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
            if output_words and output_words != ['<EOS>']: 
                hypotheses.append(output_words)
                references.append(reference_words) 
            else:
                print(f"Warning: No valid output generated for input index {i}, skipping reference.")

        except Exception as e:
            print(f"Error processing pair at index {i}: {e}")

    assert len(hypotheses) == len(references), f"Hypotheses length {len(hypotheses)} does not match references length {len(references)}"

    if hypotheses: 
        bleu_score = corpus_bleu([[ref] for ref in references], hypotheses)
        return bleu_score
    else:
        print("No valid hypotheses generated, cannot calculate BLEU score.")
        return 0.0 

In [ ]:
datasets = [train_dataloader, valid_dataloader, test_dataloader]
for i, dataloader in enumerate(datasets):
    print(f"Evaluating on {['train', 'valid', 'test'][i]} dataset")
    bleu_score = evaluate_model_on_dataloader(encoder, decoder, input_lang, output_lang, dataloader)
    print(f"BLEU score: {bleu_score:.6f}")

In [2]:
def calculate_perplexity(encoder, decoder, dataloader, input_lang, output_lang):
    encoder.eval()
    decoder.eval()
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for i, (input_tensor, target_tensor) in enumerate(dataloader):
            
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_input = target_tensor[:, :-1]  
            target_tokens = target_tensor[:, 1:]  

            decoder_input = decoder_input.float()
            target_tokens = target_tokens.float()
            # 计算交叉熵损失
            decoder_outputs, decoer_hidden,decoder_attn = decoder(encoder_outputs, encoder_hidden)
            
            loss = F.cross_entropy(decoder_outputs.reshape(-1, decoder_outputs.size(-1)), target_tensor.reshape(-1), reduction='mean')

            total_loss += loss.item()
            total_tokens += 1
            
        avg_loss = total_loss / total_tokens  # 计算平均损失

        perplexity = torch.exp(torch.tensor(avg_loss))  # 计算困惑度

    return perplexity.item()

In [ ]:
datasets = [train_dataloader, valid_dataloader, test_dataloader]

for i, dataloader in enumerate(datasets):
    print(f"Evaluating on {['train', 'valid', 'test'][i]} dataset")
    bleu_score = calculate_perplexity(encoder, decoder,  dataloader,input_lang,output_lang)
    print(f"PPL score: {bleu_score:.6f}")